In [2]:
import os
print(os.getcwd())
#os.chdir('../blocking/')
import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re
from numpy import genfromtxt

/Users/andrew/workspace/endangeredanimals/analysis


/Users/andrew/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
t = pd.read_csv("labeled.csv", encoding="ISO-8859-1", index_col=0)
t.set_index('_id')
t

,_id,ltable_id,rtable_id,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,ltable_conservation_paragraph,...,rtable_size,rtable_threats,rtable_conservation,rtable_threat_keywords,rtable_conservation_keywords,rtable_status,rtable_countries,rtable_country_count,rtable_tCount,label
0,7553756,2846,954,"blackfaced black spider monkey, chamek spider monkey, peruvian black spider monkey, peruvian spi...",Ateles,Atelidae,Terrestrial,"Bolivia, Plurinational States of; Brazil (Acre, Amazonas, Mato Grosso, RondÃ?ÃÂ´nia); Peru;",\r\r\r\r\n The major threat is subsistence and market hunting for food (with guns). An additi...,"['\n This species is confirmed, or may occur, in numerous protected areas.', <br/>, <br/>, 'B...",...,Female head-body length: 34 - 58 cm (2)Male head-body length: 42 - 50 cm (2)Female tail length: ...,The most serious threats facing this spider monkey are hunting and deforestation for agriculture...,The white-bellied spider monkey occurs in a number of protected areas in Colombia (La Macarena T...,hunting;,protected;,Critically Endangered,[],0,1,1.0
1,2107152,1762,266,white meranti,Shorea,Dipterocarpaceae,Terrestrial,Malaysia (Peninsular Malaysia);,\r\r\r\r\n The tree is slow-growing and cannot withstand logging because the reproductive cyc...,NaN,...,Trunk diameter: up to 172 cm (2),Dipterocarp forests have become amongst the most endangered in the world widely logged for use i...,Some subpopulations of this species are found in primary forest reserves where they receive vary...,,,Endangered,"['Tunisia', 'Algeria', 'Morocco']",3,0,1.0
2,2111644,6254,266,yellow meranti,Shorea,Dipterocarpaceae,Terrestrial,Brunei Darussalam; Indonesia (Sumatera); Malaysia (Sarawak);,\r\r\r\r\n It is felled for yellow meranti timber.\r\r\r\r\n\r\r\r\r\n \r\r\r\r\n,NaN,...,Trunk diameter: up to 172 cm (2),Dipterocarp forests have become amongst the most endangered in the world widely logged for use i...,Some subpopulations of this species are found in primary forest reserves where they receive vary...,,,Endangered,"['Tunisia', 'Algeria', 'Morocco']",3,0,1.0
3,4414915,6260,557,red lauan,Shorea,Dipterocarpaceae,Terrestrial,Philippines;,NaN,NaN,...,Height: up to 60 m (2),Light red meranti has been heavily exploited for its valuable timber leaving populations of this...,The light red meranti is known to occur in some forest reserves (1).,,,Endangered,['India'],1,0,1.0
4,4002060,4985,505,"armenian myotis, armenian whiskered bat",Myotis,Vespertilionidae,Terrestrial,Armenia (Armenia);,"\r\r\r\r\n Unknown, destruction of roost sites might be threat to the population.\r\r\r\r\n\r...",['\n Poorly known species with very restricted range and which requires legal protection and ...,...,5 ÃÂ¢?? 11 g (2),Recently the Indiana bat has been suffering from a decline in numbers due to a range of threats....,As well as being classified as Endangered on the IUCN Red List 2007 (1) the Indiana bat is also ...,loss;,endangered species act;,Endangered,['United States'],1,1,1.0
5,2294304,6869,289,venda cycad,Encephalartos,Zamiaceae,Terrestrial,South Africa (Limpopo Province);,\r\r\r\r\n This species is threatened due to over-collecting for ornamental purposes.\r\r\r\r...,['\n This species is listed on Appendix I of the CITES Appendices. Plants occur (or occurred)...,...,Stem length: up to 1 m (2),Over the past few decades many South African cycads have become increasingly scarce in the wild ...,There are not known to be any specific conservation measures in place for this Endangered specie...,loss;,cites;protected;,Endangered,"['Libya', 'Egypt']",2,1,1.0
6,5788640,2775,731,"shortridges capped langur, shortridges langur",Trachypithecus,Cercopithecidae,Terrestrial,China; Myanmar;,"\r\r\r\r\n These animals are hunted for food and traditional ÃÂ¢??medicine,ÃÂ¢?? sometimes ...",['\n This species is listed in CITES Appendix I. In Myanmar it is a protected species under t...,...,Male head-body length: 50 ÃÂ¢?? 65 cm (2)

In [13]:
for c in t.columns:
    print(c)

_id
ltable_id
rtable_id
ltable_name
ltable_genus
ltable_family
ltable_ecology
ltable_countries
ltable_threat_paragraph
ltable_conservation_paragraph
ltable_pop_trend
ltable_status
ltable_country_count
ltable_scientific_name
rtable_scientific_name
rtable_name
rtable_kingdom
rtable_phylum
rtable_class
rtable_order
rtable_family
rtable_genus
rtable_size
rtable_threats
rtable_conservation
rtable_threat_keywords
rtable_conservation_keywords
rtable_status
rtable_countries
rtable_country_count
rtable_tCount
label


In [21]:
print('Number of tuples:', len(t))
print('Number of unique rtable values', len(t.rtable_scientific_name.unique()))
print('Number of unique genuses:', len(t.ltable_genus.unique()))
print('Number of unique sizes:', len(t.rtable_size.unique()))

Number of tuples: 664
Number of unique rtable values 186
Number of unique genuses: 134
Number of unique sizes: 178


In [22]:
t.rtable_size.unique()

array([ 'Female head-body length: 34 - 58 cm (2)Male head-body length: 42 - 50 cm (2)Female tail length: 61 - 88 cm (2)Male tail length: 69.5 - 82 cm (2)Female weight: 7.5 - 10.4 kg (2)Male weight: 7.3 - 9.8 kg (2)',
       'Trunk diameter: up to 172 cm (2)', 'Height: up to 60 m (2)',
       '5 Ã\x83Â¢?? 11 g (2)', 'Stem length: up to 1 m (2)',
       'Male head-body length: 50 Ã\x83Â¢?? 65 cm (2)Female head-body length: 45 Ã\x83Â¢?? 60 cm (2)Male tail length: 67 Ã\x83Â¢?? 85 cm(2)Female tail length: 62 Ã\x83Â¢?? 82 cm (2)Male weight: 3.4 Ã\x83Â¢?? 9.4 kg (2)Female weight: 3.1 Ã\x83Â¢?? 9 kg (2)',
       'Extended length: up to 90 mm (2)',
       'Female snout-vent length: up to 34 mm (2)',
       'Max height: 2-3 m (2)',
       'Male length: 22 Ã\x83Â¢?? 25 mm (2)Female length: 25 Ã\x83Â¢?? 30 mm (2)',
       'Head-body length: 8.4 cm (2)Tail length: 8.8 cm (2)',
       'Maximum length: 8.1 cm (2)',
       'Height: 3 - 7 m (2)Trunk diameter: c. 20 cm (2)',
       'Length: 5.6 - 10.5 c

In [46]:
u = t.rtable_size.unique()
lengths = [l for l in u if isinstance(l, str) and 'length' in l]
print('number of lengths:', len(lengths))
notlengths = [l for l in u if isinstance(l, str) and 'length' not in l]
weights = [l for l in u if isinstance(l, str) and (' g ' in l or ' kg ' in l)]
print('Number of weights', len(weights))
print()
print()
print()
for w in weights:
    
    match = None
    # find last occurrance of "<number> g" or "<number> kg"
    for match in re.finditer(r"\d (k)?g", w):
        pass
    #only keep the digit
    m = float(re.sub(r'\D', "", match.group()))
    if 'k' in match.group():
        m *= 1000

    print(m)
    print()

# for size in t.rtable_size.unique():
#     print(size)
#     print()


add weight column


weightColumn = []

# replace name fields with list of nick/common names
for index, row in t.iterrows():
    size = row['rtable_size']
    if not size:
        continue
        
    #if this is a weight
    if isinstance(size, str) and (' g ' in l or ' kg ' in l):
        
    else:
        weightColumn.append(None)
        

    if indCommonNameStart !=-1:        
        # extract full common/nick name
        indCommonNameEnd = nameString.find(")")
        commonName = nameString[indCommonNameStart+1:indCommonNameEnd]
        sciName = nameString[:indCommonNameStart-1].strip()
        df.loc[index, 'animalName'] = commonName
        sciNameColumn.append(sciName)
    else:
        error = error + 1
        df.loc[index, 'animalName'] = ""
        sciNameColumn.append(nameString)

df['scientific_name'] = sciNameColumn
print(error)        
df.head()

number of lengths: 64
Number of weights 65



8000.0

1.0

9000.0

0.0

3.0

5000.0

5000.0

2000.0

5000.0

6000.0

4000.0

0.0

7000.0

0.0

0.0

0.0

2000.0

3000.0

4000.0

5.0

3000.0

0.0

1000.0

7000.0

5000.0

3000.0

5.0

0.0

5000.0

9000.0

4000.0

5000.0

4000.0

0.0

0.0

5000.0

9.0

9.0

0.0

0.0

8000.0

5.0

4.0

8.0

6.0

7.0

9000.0

4000.0

5000.0

3000.0

0.0

6000.0

0.0

5000.0

6000.0

3000.0

4000.0

0.0

0.0

5000.0

5000.0

8000.0

0.0

0.0

4.0

